In [1]:
import pandas as pd
import numpy as np

C:\Users\koles\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [17]:
df_train = pd.read_csv('data/train_row.csv')
df_train.head()

,category,text
0,extreme,Ледник Пасторури это цирковой ледник расположе...
1,martial_arts,Главные участники предстоящего Betokenoid 274 ...
2,extreme,Ttokenoid Btokenoid – карта с которой можно не...
3,autosport,В Сильверстоуне произошли крупные обновления а...
4,extreme,На протяжении более чем 30 лет Вестсайд являет...


In [22]:
df_train['category'].value_counts()

category
autosport       608
athletics       603
boardgames      598
esport          591
tennis          590
martial_arts    584
extreme         572
basketball      570
hockey          563
motosport       562
volleyball      560
football        555
winter_sport    544
Name: count, dtype: int64

In [24]:
!pip install pymorphy2

     -------------------------------------- 55.5/55.5 kB 411.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.2/8.2 MB 4.5 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13775 sha256=921c9c3715bcfac8aec1e172dc4e9e6c4e5726679d8a2be86b46d879557dc45c
  Stored in directory: c:\users\koles\appdata\local\pip\cache\wheels\7c\d7\8d\2156234738063e3d4a39ba77dc677046100e62766b53807189
Successfully built docopt


In [39]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    return ' '.join(lemmatized_words)

In [29]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import balanced_accuracy_score
import optuna
from sklearn.preprocessing import LabelEncoder

In [43]:
df_train['normalized_text'] = df_train['text'].apply(lemmatize_text)

In [67]:
X = df_train['normalized_text']
y = df_train['category']
label_enc = LabelEncoder()
y_encoded = label_enc.fit_transform(y)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [69]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [70]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

In [93]:
model = svm.SVC(kernel='rbf')
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_val_tfidf)
accuracy = balanced_accuracy_score(y_true=y_val, y_pred=y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7456013073410548


In [96]:
def objective(trial):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf']),
        'C': trial.suggest_float('C', 0.001, 1000, log=True)
    }
    model = svm.SVC(**params)
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_val_tfidf)
    return balanced_accuracy_score(y_true=y_val, y_pred=y_pred)

In [97]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-02-16 15:28:09,692] A new study created in memory with name: no-name-18350a5f-b63c-495e-b75c-b0a276570d83
[I 2025-02-16 15:28:22,432] Trial 0 finished with value: 0.7603545564872923 and parameters: {'kernel': 'rbf', 'C': 34.249519704949236}. Best is trial 0 with value: 0.7603545564872923.
[I 2025-02-16 15:28:34,973] Trial 1 finished with value: 0.5155522790306216 and parameters: {'kernel': 'poly', 'C': 142.01689474529698}. Best is trial 0 with value: 0.7603545564872923.
[I 2025-02-16 15:28:47,699] Trial 2 finished with value: 0.7611394858436502 and parameters: {'kernel': 'rbf', 'C': 3.818582911025586}. Best is trial 2 with value: 0.7611394858436502.
[I 2025-02-16 15:29:00,375] Trial 3 finished with value: 0.7603545564872923 and parameters: {'kernel': 'rbf', 'C': 26.790949204465655}. Best is trial 2 with value: 0.7611394858436502.
[I 2025-02-16 15:29:12,083] Trial 4 finished with value: 0.5918717016284805 and parameters: {'kernel': 'linear', 'C': 0.18686071558459752}. Best is tr

In [99]:
study.best_params

{'kernel': 'linear', 'C': 0.9806781702720403}

In [100]:
model = svm.SVC(**study.best_params)
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_val_tfidf)
accuracy = balanced_accuracy_score(y_pred=y_pred, y_true=y_val)
print(f"Accuracy: {accuracy}")


Accuracy: 0.7717341642059622


In [102]:
y_pred = model.predict(X_test_tfidf)
accuracy = balanced_accuracy_score(y_pred=y_pred, y_true=y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7788368679266922


In [104]:
import pickle

with open('models/svm_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

with open('models/tfidf_vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

In [108]:
df_test = pd.read_csv('data/test_row.csv')
df_test

,text
0,Наступила та самая неделя Nattokenoid Setokeno...
1,Тоня Сегодня я записываю свое первое видео на ...
2,Mtokenoid Games дисквалифицировали с мейджора ...
3,Скотт Джурек Почему я бегаю Команда tokentoken...
4,Пьер Гасли Гонка в Монреале будет особенно сло...
...,...
2495,КонкурсГРаНИ tokenoid Полторашка В этот раз те...
2496,когда понял что тебя кинули но пооучил поддерж...
2497,С ЛеБроном что то не так надеемся что ничего с...
2498,Хочешь принять участие в зимнем дрифте? Есть Ж...


In [109]:
df_test['normalized_text'] = df_test['text'].apply(lemmatize_text)

In [111]:
X_submission = df_test['normalized_text']
X_submission = vectorizer.transform(X_submission)

In [112]:
y_submission = model.predict(X_submission)

In [114]:
y_submission = label_enc.inverse_transform(y_submission)

In [117]:
submission = pd.DataFrame({'category': y_submission})

In [119]:
submission.to_csv('submissions/svm.csv', index=False)